# GBP/USD M15 2025 — MACD Intraday Backtest

This notebook explores MACD-based intraday trading signals on GBP/USD M15 data (2025).  
All trades open and close within the same day — **no overnight positions**.

---

## MACD Overview

`ta.macd(close, fast=12, slow=26, signal=9)` returns:
- **MACD line** — fast EMA minus slow EMA
- **Signal line** — EMA of the MACD line
- **Histogram** — MACD minus signal

## Strategy: MACD Crossover Intraday

| | Rule |
|---|---|
| **Entry window** | 08:00–15:00 UTC (London/NY session) |
| **Long signal** | MACD line crosses above signal line |
| **Short signal** | MACD line crosses below signal line |
| **Exit** | 17:00 UTC time exit (or SL/TP if set) |
| **Filter** | One trade per day max (no re-entry after exit) |

**Rationale**: Classic MACD crossover restricted to active London/NY hours to avoid noise in Asian session.

---

**Data**: `GBP_USD_M15_20250101_20251231.csv` (~24,773 candles)

## 1. Setup and Imports

In [ ]:
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from backtesting import Backtest, Strategy
import pandas_ta as ta
from pathlib import Path

pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_rows', 100)
plt.style.use('seaborn-v0_8-darkgrid')

print('\u2713 Imports successful')

## 2. Configuration

In [ ]:
PAIR         = 'GBP_USD'
GRANULARITY  = 'M15'
FROM_DATE    = '20250101'
TO_DATE      = '20251231'

INITIAL_CASH = 10_000
COMMISSION   = 0.0001   # ~1 pip round-trip
EXIT_HOUR    = 17       # UTC hour to close all positions

MACD_FAST   = 12
MACD_SLOW   = 26
MACD_SIGNAL = 9

DATA_DIR = Path('../data/historical')
data_path = DATA_DIR / PAIR / f'{PAIR}_{GRANULARITY}_{FROM_DATE}_{TO_DATE}.csv'

print(f'Pair        : {PAIR}')
print(f'Timeframe   : {GRANULARITY}')
print(f'Period      : {FROM_DATE} \u2013 {TO_DATE}')
print(f'Capital     : ${INITIAL_CASH:,.0f}')
print(f'Commission  : {COMMISSION}')
print(f'Exit Hour   : {EXIT_HOUR}:00 UTC')
print(f'MACD Params : fast={MACD_FAST}, slow={MACD_SLOW}, signal={MACD_SIGNAL}')
print(f'Data path   : {data_path}  {"\u2713" if data_path.exists() else "\u2717 MISSING"}')

## 3. Load Data

In [ ]:
def load_data(path: Path) -> pd.DataFrame:
    """Load OANDA CSV, return DataFrame formatted for backtesting.py."""
    df = pd.read_csv(path, comment='#', parse_dates=['time'], index_col='time')
    df.index = pd.to_datetime(df.index, utc=True)
    df.columns = [c.capitalize() for c in df.columns]
    return df[['Open', 'High', 'Low', 'Close', 'Volume']]


df = load_data(data_path)

print(f'Candles    : {len(df):,}')
print(f'Date range : {df.index.min()} \u2192 {df.index.max()}')
print(f'Columns    : {list(df.columns)}')
df.head()

## 4. Exploratory: MACD on GBP/USD M15

Compute MACD on full dataset and visualise crossover characteristics.

In [ ]:
# Compute MACD on full dataset
macd_df = ta.macd(df['Close'], fast=MACD_FAST, slow=MACD_SLOW, signal=MACD_SIGNAL)
macd_col = f'MACD_{MACD_FAST}_{MACD_SLOW}_{MACD_SIGNAL}'
signal_col = f'MACDs_{MACD_FAST}_{MACD_SLOW}_{MACD_SIGNAL}'
hist_col = f'MACDh_{MACD_FAST}_{MACD_SLOW}_{MACD_SIGNAL}'

df['MACD'] = macd_df[macd_col]
df['MACD_Signal'] = macd_df[signal_col]
df['MACD_Hist'] = macd_df[hist_col]

# Detect crossovers
df['cross_up'] = (df['MACD'].shift(1) <= df['MACD_Signal'].shift(1)) & (df['MACD'] > df['MACD_Signal'])
df['cross_down'] = (df['MACD'].shift(1) >= df['MACD_Signal'].shift(1)) & (df['MACD'] < df['MACD_Signal'])
df['crossover'] = df['cross_up'] | df['cross_down']

print(f'MACD NaN count (warmup): {df["MACD"].isna().sum()}')
print(f'Total crossovers: {df["crossover"].sum()}')
print(f'  - Bullish (up) : {df["cross_up"].sum()}')
print(f'  - Bearish (down): {df["cross_down"].sum()}')

# Plot sample 2-week window
sample_start = '2025-03-03'
sample_end   = '2025-03-14'
s = df.loc[sample_start:sample_end]

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(16, 8), sharex=True,
                                gridspec_kw={'height_ratios': [2, 1]})

ax1.plot(s.index, s['Close'], color='black', linewidth=1)
ax1.set_title(f'{PAIR} M15 Price — {sample_start} to {sample_end}')
ax1.set_ylabel('Price')

ax2.plot(s.index, s['MACD'], color='blue', linewidth=1, label='MACD')
ax2.plot(s.index, s['MACD_Signal'], color='orange', linewidth=1, label='Signal')
colors = ['green' if v >= 0 else 'red' for v in s['MACD_Hist']]
ax2.bar(s.index, s['MACD_Hist'], color=colors, alpha=0.5, width=0.008, label='Histogram')
ax2.axhline(0, color='black', linewidth=0.5)
ax2.set_title('MACD (12, 26, 9)')
ax2.set_ylabel('MACD Value')
ax2.legend(fontsize=8)

plt.tight_layout()
plt.show()

# Daily crossover frequency
cross_bars = df[df['crossover']].copy()
daily_crosses = cross_bars.groupby(cross_bars.index.date).size()
print(f'\nAvg crossovers per day: {daily_crosses.mean():.1f}')
print(f'Median crossovers per day: {daily_crosses.median():.0f}')

# Hourly distribution of crossovers
hourly_crosses = cross_bars.groupby(cross_bars.index.hour).size()

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 4))

ax1.hist(daily_crosses.values, bins=range(0, daily_crosses.max() + 2), color='steelblue',
         edgecolor='white', alpha=0.8, align='left')
ax1.axvline(daily_crosses.mean(), color='red', linestyle='--',
            label=f'Mean: {daily_crosses.mean():.1f}')
ax1.set_title('Distribution of Daily Crossover Count')
ax1.set_xlabel('Crossovers per day')
ax1.set_ylabel('Days')
ax1.legend()

ax2.bar(hourly_crosses.index, hourly_crosses.values, color='steelblue')
ax2.axvspan(7.5, 15.5, alpha=0.15, color='green', label='Entry window (08-15)')
ax2.set_title('Hourly Distribution of MACD Crossovers')
ax2.set_xlabel('Hour UTC')
ax2.set_ylabel('Crossover count')
ax2.legend(fontsize=8)

plt.tight_layout()
plt.show()

## 5. Signal Quality: Raw MACD Crossover Analysis

Per-day analysis (no backtesting.py): for each day, find the first MACD crossover in the 08:00–15:00 UTC window,  
enter at that bar's close, exit at 17:00 open. Compute win rate, avg P&L, profit factor.

In [ ]:
raw = df.copy().reset_index()
raw.columns = [c.lower() for c in raw.columns]
raw['hour']   = raw['time'].dt.hour
raw['minute'] = raw['time'].dt.minute
raw['date']   = raw['time'].dt.date

signal_rows = []
for date, day in raw.groupby('date'):
    day = day.sort_values('time')

    # Find first crossover in 08:00-15:00 window
    window = day[(day['hour'] >= 8) & (day['hour'] <= 15)]
    crosses = window[window['cross_up'] | window['cross_down']]

    if len(crosses) == 0:
        continue

    first = crosses.iloc[0]
    direction = 1 if first['cross_up'] else -1

    # Exit at 17:00
    exit_bar = day[(day['hour'] == EXIT_HOUR) & (day['minute'] == 0)]
    if len(exit_bar) == 0:
        continue

    entry_px = first['close']
    exit_px  = exit_bar.iloc[0]['open']
    pnl_pips = direction * (exit_px - entry_px) * 10_000

    signal_rows.append({
        'date':      date,
        'dow':       pd.Timestamp(date).day_name(),
        'signal':    'Long' if direction == 1 else 'Short',
        'entry_hour': first['hour'],
        'entry_px':  entry_px,
        'exit_px':   exit_px,
        'pnl_pips':  round(pnl_pips, 1),
        'win':       pnl_pips > 0,
    })

sig_df = pd.DataFrame(signal_rows)

print(f'Total days with signal : {len(sig_df)}')
print(f'  - Long signals       : {(sig_df["signal"] == "Long").sum()}')
print(f'  - Short signals      : {(sig_df["signal"] == "Short").sum()}')
print(f'Win rate               : {sig_df["win"].mean()*100:.1f}%')
print(f'Avg P&L                : {sig_df["pnl_pips"].mean():.1f} pips')
print(f'Total P&L              : {sig_df["pnl_pips"].sum():.0f} pips')

# Profit factor
gross_profit = sig_df[sig_df['pnl_pips'] > 0]['pnl_pips'].sum()
gross_loss   = abs(sig_df[sig_df['pnl_pips'] < 0]['pnl_pips'].sum())
pf = gross_profit / gross_loss if gross_loss > 0 else float('inf')
print(f'Profit factor          : {pf:.2f}')

# By day of week
dow_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
dow_stats = sig_df.groupby('dow').agg(
    trades=('pnl_pips', 'count'),
    win_rate=('win', 'mean'),
    avg_pnl=('pnl_pips', 'mean'),
    total_pnl=('pnl_pips', 'sum'),
).reindex(dow_order).round(2)
dow_stats['win_rate'] = (dow_stats['win_rate'] * 100).round(1)
print('\nBy day of week:')
display(dow_stats)

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

axes[0].plot(range(len(sig_df)), sig_df['pnl_pips'].cumsum(), color='steelblue')
axes[0].axhline(0, color='black', linewidth=0.8, linestyle='--')
axes[0].set_title('MACD Crossover — Cumulative P&L (pips)')
axes[0].set_xlabel('Trade #')
axes[0].set_ylabel('Cumulative pips')

wr_dow = sig_df.groupby('dow')['win'].mean().reindex(dow_order) * 100
axes[1].bar(wr_dow.index, wr_dow.values,
            color=['green' if v > 50 else 'red' for v in wr_dow.values])
axes[1].axhline(50, color='black', linewidth=0.8, linestyle='--')
axes[1].set_title('Win Rate by Day of Week (%)')
axes[1].set_ylabel('Win Rate (%)')
axes[1].set_xticklabels([d[:3] for d in dow_order])

# Entry hour distribution
entry_hours = sig_df['entry_hour'].value_counts().sort_index()
axes[2].bar(entry_hours.index, entry_hours.values, color='steelblue')
axes[2].set_title('Entry Hour Distribution')
axes[2].set_xlabel('Hour UTC')
axes[2].set_ylabel('Count')

plt.tight_layout()
plt.show()

## 6. MACD Helper Functions + Strategy Classes

Since `self.I()` needs scalar-returning functions, we use helper functions  
(same pattern as `bb_upper`/`bb_lower` in `rsi_mean_reversion.py`).

In [ ]:
def macd_line(s, fast, slow, signal):
    return ta.macd(s, fast=fast, slow=slow, signal=signal).iloc[:, 0].values


def macd_signal_line(s, fast, slow, signal):
    return ta.macd(s, fast=fast, slow=slow, signal=signal).iloc[:, 2].values


class MACDCrossoverIntradayBase(Strategy):
    """
    MACD crossover during London/NY session (08:00-15:00 UTC).
    Exit at exit_hour UTC. No SL/TP — time exit only.
    """
    exit_hour   = 17
    entry_start = 8
    entry_end   = 15
    macd_fast   = 12
    macd_slow   = 26
    macd_signal = 9

    def init(self):
        close = pd.Series(self.data.Close)
        self.macd = self.I(macd_line, close, self.macd_fast, self.macd_slow, self.macd_signal)
        self.sig  = self.I(macd_signal_line, close, self.macd_fast, self.macd_slow, self.macd_signal)

    def next(self):
        bar_time = self.data.index[-1]

        # Exit at exit_hour
        if bar_time.hour == self.exit_hour and bar_time.minute == 0 and self.position:
            self.position.close()
            return

        if self.position:
            return

        # Entry window
        if not (self.entry_start <= bar_time.hour <= self.entry_end):
            return

        # Crossover detection
        if self.macd[-2] <= self.sig[-2] and self.macd[-1] > self.sig[-1]:
            self.buy()
        elif self.macd[-2] >= self.sig[-2] and self.macd[-1] < self.sig[-1]:
            self.sell()


class MACDCrossoverIntraday(Strategy):
    """
    MACD crossover with stop loss and take profit.
    Time exit at exit_hour as backstop.
    """
    exit_hour   = 17
    entry_start = 8
    entry_end   = 15
    macd_fast   = 12
    macd_slow   = 26
    macd_signal = 9
    sl_pips     = 20
    tp_pips     = 40

    def init(self):
        close = pd.Series(self.data.Close)
        self.macd = self.I(macd_line, close, self.macd_fast, self.macd_slow, self.macd_signal)
        self.sig  = self.I(macd_signal_line, close, self.macd_fast, self.macd_slow, self.macd_signal)

    def next(self):
        bar_time = self.data.index[-1]

        # Exit at exit_hour
        if bar_time.hour == self.exit_hour and bar_time.minute == 0 and self.position:
            self.position.close()
            return

        if self.position:
            return

        # Entry window
        if not (self.entry_start <= bar_time.hour <= self.entry_end):
            return

        # Crossover detection with SL/TP
        pip = 0.0001
        c = self.data.Close[-1]
        if self.macd[-2] <= self.sig[-2] and self.macd[-1] > self.sig[-1]:
            self.buy(sl=c - self.sl_pips * pip, tp=c + self.tp_pips * pip)
        elif self.macd[-2] >= self.sig[-2] and self.macd[-1] < self.sig[-1]:
            self.sell(sl=c + self.sl_pips * pip, tp=c - self.tp_pips * pip)


print('\u2713 Strategy classes defined')
print('  - MACDCrossoverIntradayBase : time exit only')
print('  - MACDCrossoverIntraday     : with SL/TP')

## 7. Backtest: Base + SL/TP

In [ ]:
# Drop exploratory columns before backtesting
bt_df = df[['Open', 'High', 'Low', 'Close', 'Volume']].copy()

# Base strategy (time exit)
bt_base = Backtest(bt_df, MACDCrossoverIntradayBase, cash=INITIAL_CASH, commission=COMMISSION)
stats_base = bt_base.run()

print('=== MACD Base (time exit) ===')
print(f"Return          : {stats_base['Return [%]']:.2f}%")
print(f"Sharpe Ratio    : {stats_base['Sharpe Ratio']:.3f}")
print(f"Win Rate        : {stats_base['Win Rate [%]']:.1f}%")
print(f"Max Drawdown    : {stats_base['Max. Drawdown [%]']:.2f}%")
print(f"# Trades        : {stats_base['# Trades']}")
print()

# SL/TP strategy
bt_sltp = Backtest(bt_df, MACDCrossoverIntraday, cash=INITIAL_CASH, commission=COMMISSION)
stats_sltp = bt_sltp.run()

print('=== MACD with SL/TP (default: 20/40 pips) ===')
print(f"Return          : {stats_sltp['Return [%]']:.2f}%")
print(f"Sharpe Ratio    : {stats_sltp['Sharpe Ratio']:.3f}")
print(f"Win Rate        : {stats_sltp['Win Rate [%]']:.1f}%")
print(f"Max Drawdown    : {stats_sltp['Max. Drawdown [%]']:.2f}%")
print(f"# Trades        : {stats_sltp['# Trades']}")

## 8. Optimisation

In [ ]:
# Optimise base strategy MACD parameters
print('Optimising MACD parameters (this may take a few minutes)...')

bt_base_opt = Backtest(bt_df, MACDCrossoverIntradayBase, cash=INITIAL_CASH, commission=COMMISSION)
stats_base_opt = bt_base_opt.optimize(
    macd_fast=range(8, 16, 2),
    macd_slow=range(20, 32, 2),
    macd_signal=range(7, 12),
    exit_hour=range(15, 20),
    maximize='Sharpe Ratio',
    constraint=lambda p: p.macd_slow > p.macd_fast + 6,
    return_heatmap=False,
)

print('\n' + '=' * 60)
print('MACD BASE OPTIMISED RESULTS')
print('=' * 60)
print(f"Return          : {stats_base_opt['Return [%]']:.2f}%")
print(f"Sharpe Ratio    : {stats_base_opt['Sharpe Ratio']:.3f}")
print(f"Win Rate        : {stats_base_opt['Win Rate [%]']:.1f}%")
print(f"Max Drawdown    : {stats_base_opt['Max. Drawdown [%]']:.2f}%")
print(f"# Trades        : {stats_base_opt['# Trades']}")
print(f"Best MACD fast  : {stats_base_opt._strategy.macd_fast}")
print(f"Best MACD slow  : {stats_base_opt._strategy.macd_slow}")
print(f"Best MACD signal: {stats_base_opt._strategy.macd_signal}")
print(f"Best exit hour  : {stats_base_opt._strategy.exit_hour}:00 UTC")

In [ ]:
# Optimise SL/TP strategy
print('Optimising SL/TP parameters (this may take a few minutes)...')

bt_sltp_opt = Backtest(bt_df, MACDCrossoverIntraday, cash=INITIAL_CASH, commission=COMMISSION)
stats_sltp_opt = bt_sltp_opt.optimize(
    sl_pips=range(10, 41, 5),
    tp_pips=range(20, 81, 10),
    exit_hour=range(15, 20),
    maximize='Sharpe Ratio',
    constraint=lambda p: p.tp_pips > p.sl_pips,
    return_heatmap=False,
)

print('\n' + '=' * 60)
print('MACD SL/TP OPTIMISED RESULTS')
print('=' * 60)
print(f"Return          : {stats_sltp_opt['Return [%]']:.2f}%")
print(f"Sharpe Ratio    : {stats_sltp_opt['Sharpe Ratio']:.3f}")
print(f"Win Rate        : {stats_sltp_opt['Win Rate [%]']:.1f}%")
print(f"Max Drawdown    : {stats_sltp_opt['Max. Drawdown [%]']:.2f}%")
print(f"# Trades        : {stats_sltp_opt['# Trades']}")
print(f"Best SL         : {stats_sltp_opt._strategy.sl_pips} pips")
print(f"Best TP         : {stats_sltp_opt._strategy.tp_pips} pips")
print(f"Best exit hour  : {stats_sltp_opt._strategy.exit_hour}:00 UTC")

## 9. Equity Curves

In [ ]:
eq_base     = stats_base['_equity_curve']
eq_sltp     = stats_sltp['_equity_curve']
eq_base_opt = stats_base_opt['_equity_curve']
eq_sltp_opt = stats_sltp_opt['_equity_curve']

fig = go.Figure()
fig.add_trace(go.Scatter(x=eq_base.index, y=eq_base['Equity'],
                         name='Base (time exit)', line=dict(width=1.5)))
fig.add_trace(go.Scatter(x=eq_sltp.index, y=eq_sltp['Equity'],
                         name='SL/TP Default', line=dict(width=1.5)))
fig.add_trace(go.Scatter(x=eq_base_opt.index, y=eq_base_opt['Equity'],
                         name='Base Optimised', line=dict(width=2, dash='dot')))
fig.add_trace(go.Scatter(x=eq_sltp_opt.index, y=eq_sltp_opt['Equity'],
                         name='SL/TP Optimised', line=dict(width=2, dash='dash')))
fig.add_hline(y=INITIAL_CASH, line_dash='dash', line_color='gray', annotation_text='Initial capital')
fig.update_layout(
    title='MACD Crossover Intraday — Equity Curves',
    xaxis_title='Date', yaxis_title='Equity ($)',
    height=500, hovermode='x unified'
)
fig.show()

## 10. Strategy Comparison

In [ ]:
METRICS = ['Return [%]', 'Sharpe Ratio', 'Win Rate [%]', 'Max. Drawdown [%]', '# Trades']

variants = {
    'MACD Base':         stats_base,
    'MACD SL/TP':        stats_sltp,
    'MACD Base Opt':     stats_base_opt,
    'MACD SL/TP Opt':    stats_sltp_opt,
}

comparison = pd.DataFrame({
    name: {m: round(stats[m], 2) for m in METRICS}
    for name, stats in variants.items()
})

print('Strategy Comparison \u2014 GBP/USD MACD Intraday M15 2025\n')
display(comparison)

## 11. Conclusions

### Strategy Summary

| Strategy | Return % | Sharpe | Win Rate | Max DD | # Trades |
|----------|----------|--------|----------|--------|----------|
| MACD Base | *(fill)* | *(fill)* | *(fill)* | *(fill)* | *(fill)* |
| MACD SL/TP | *(fill)* | *(fill)* | *(fill)* | *(fill)* | *(fill)* |
| MACD Base Opt | *(fill)* | *(fill)* | *(fill)* | *(fill)* | *(fill)* |
| MACD SL/TP Opt | *(fill)* | *(fill)* | *(fill)* | *(fill)* | *(fill)* |

### Key Findings

| Question | Answer |
|----------|--------|
| Which variant has the best Sharpe Ratio? | *(fill after running)* |
| Does adding SL/TP improve or hurt vs time exit? | *(fill after running)* |
| What are the optimal MACD parameters? | *(fill after running)* |
| What is the optimal SL/TP combination? | *(fill after running)* |
| Which entry hours produce the most crossovers? | *(fill after running)* |

### Strengths
- **MACD crossover** is a well-understood momentum signal
- Entry restricted to London/NY session avoids low-liquidity noise
- Fully intraday: zero overnight exposure, no financing risk
- One trade per day: simple position management

### Weaknesses and Risks
- MACD is a lagging indicator — entries may come after a significant move
- In-sample only — **must validate on out-of-sample data** (e.g., 2024 data)
- Multiple crossovers per day (whipsaw) can degrade signal quality
- Optimised parameters may be overfit to 2025 — treat with caution

### Recommended Next Steps
1. **Out-of-sample test**: run on 2024 GBP/USD data
2. **Histogram filter**: only trade crossovers when histogram shows increasing momentum
3. **Combine with trend filter**: add H1 EMA200 to filter against-trend entries
4. **Compare with notebook 05**: ARB/LMM strategies on same data
5. **Walk-forward analysis**: rolling 3-month train / 1-month test windows

---
**Reminder**: In-sample backtest results are optimistic. Past performance does not guarantee future results.